# RAG Agent Creation Tutorial

## Overview
This notebook demonstrates how to create a **Retrieval-Augmented Generation (RAG) agent** using the Hyland Agent Platform API. You'll learn to:

1. **Authenticate** with the Hyland API using OAuth 2.0 client credentials
2. **Configure** a RAG agent with specific parameters
3. **Create** the agent via REST API calls
4. **Understand** the response and next steps

## Prerequisites
- Valid Hyland client credentials (`CLIENT_ID` and `CLIENT_SECRET`)
- Basic understanding of REST APIs and JSON
- Familiarity with Python (helpful but not required)

## Instructions
1. **Run cells sequentially** from top to bottom
2. **Read the explanations** before each code section
3. **Enter your credentials** when prompted (they will be hidden)
4. **Check the output** after each cell execution

---

## Step 1: Import Libraries and Define Authentication Function

The cell below imports the necessary Python libraries and defines a helper function for OAuth authentication.

**What this does:**
- **`json`**: For parsing API responses
- **`os`**: For accessing environment variables
- **`urllib`**: For making HTTP requests to the authentication endpoint
- **`requests`**: For making the main API calls
- **`getpass`**: For securely entering passwords (hides input)

**The `get_token()` function:**
- Takes client credentials as input
- Makes a POST request to Hyland's OAuth token endpoint
- Returns an access token for API authorization
- Handles errors gracefully

In [ ]:
import json
import urllib
import urllib.parse
import urllib.request
import requests
from getpass import getpass

def get_token(
    data: dict[str, str],
) -> str | None:
    encoded_data = urllib.parse.urlencode(data).encode("utf-8")

    headers_for_auth_request = {"Content-Type": "application/x-www-form-urlencoded"}

    auth_url = "https://auth.iam.dev.experience.hyland.com/idp/connect/token"
    print(f"Getting token from {auth_url}")
    try:
        req = urllib.request.Request(auth_url, data=encoded_data, headers=headers_for_auth_request, method="POST")

        with urllib.request.urlopen(req) as response_from_auth:            
            response_data = json.loads(response_from_auth.read().decode("utf-8"))
            access_token = response_data.get("access_token")
            print(f"access_token: {access_token}")
            return access_token

    except Exception as e:
        print(f"Failed to get token: {str(e)}")

    return None

## Step 2: Authenticate with Your Credentials

Now you'll enter your Hyland API credentials to get authorized access.

**What happens when you run this cell:**
1. **Environment Check**: First checks if credentials are available as environment variables
2. **Secure Input**: If not found, prompts you to enter them manually (input will be hidden)
3. **Validation**: Shows whether credentials were found or entered

**🔐 Security Note:** Your credentials are never displayed or stored in the notebook output.

**📝 If prompted, enter:**
- Your **Client ID** (looks like a UUID)
- Your **Client Secret** (a longer string)

In [ ]:
# Authentication - Enter your Hyland API credentials
print("Please enter your Hyland API credentials")
print("Your input will be hidden for security")

client_id = getpass("Enter your CLIENT_ID: ")
client_secret = getpass("Enter your CLIENT_SECRET: ")

if client_id and client_secret:
    print("Credentials entered successfully!")
else:
    print("Missing credentials - please run this cell again")

## Step 3: Get Access Token

This cell exchanges your credentials for an access token using OAuth 2.0 client credentials flow.

**What this does:**
- Calls the `get_token()` function with your credentials
- Uses the "client_credentials" grant type for server-to-server authentication
- Requests access to "hxp environment_authorization" scope
- Returns a temporary access token (typically valid for 1 hour)

**Expected output:** You should see the token endpoint URL and a success message with your access token.

In [ ]:
access_token = get_token({
        "client_id": client_id,
        "grant_type": "client_credentials",
        "client_secret": client_secret,
        "scope": "hxp environment_authorization"
    })

## Step 4: Prepare HTTP Headers

This cell sets up the HTTP headers needed for the API request to create your RAG agent.

**Headers explained:**
- **`Content-Type`**: Tells the server we're sending JSON data
- **`Accept`**: Tells the server we expect JSON response
- **`Authorization`**: Includes your Bearer token for authentication

**Note:** The `f'Bearer {access_token}'` syntax inserts your token from the previous step.

In [ ]:
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'Authorization': f'Bearer {access_token}'
}

## Step 5: Configure Your RAG Agent

The next cell defines the configuration for your RAG agent. This JSON payload specifies how your agent will behave.

**Key Configuration Options:**
- **`name`**: A friendly name for your agent ("DocumentHelper")
- **`description`**: What your agent does ("Document assistant")
- **`agentType`**: Set to "rag" for Retrieval-Augmented Generation
- **`limit`**: Maximum number of documents to retrieve (25)
- **`llm_model_id`**: The AI model to use (Amazon Nova Micro)
- **`system_prompt`**: Instructions that guide how the AI responds
- **`temperature`**: Controls creativity (0.7 = balanced)
- **`max_tokens`**: Maximum response length (4000 tokens)

**💡 Tip:** You can modify these values to customize your agent's behavior before running the cell.

In [ ]:
payload = {
    "name": "DocumentHelper",
    "description": "Document assistant",
    "agentType": "rag",
    "notes": "Initial version - shows all configuration options",
    "config": {
        "filter_value": {},
        "limit": 25,
        "llm_model_id": "amazon.nova-micro-v1:0",
        "system_prompt": "Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer:",
        "inference_config": {
            "temperature": 0.7,
            "max_tokens": 4000
        }
    }
}

## Step 6: Create Your RAG Agent

This is the main API call that creates your RAG agent on the Hyland platform.

**What this does:**
- Makes a POST request to the agents endpoint
- Sends your agent configuration (from Step 5) as JSON
- Includes authentication headers (from Step 4)
- Returns the creation result

**Expected responses:**
- **Status 201**: ✅ Success - Agent created successfully
- **Status 400**: ❌ Bad Request - Check your configuration
- **Status 401**: ❌ Unauthorized - Check your credentials
- **Status 403**: ❌ Forbidden - Check your permissions

**The response will include:**
- Your new agent's unique ID
- Confirmation of the configuration
- Any validation messages

In [ ]:
response = requests.post("https://api.agents.ai.dev.experience.hyland.com/agent-platform/v1/agents", json=payload, headers=headers)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.text}")

# Extract agent ID if creation was successful
agent_id = None
if response.status_code == 201:
    try:
        response_data = response.json()
        agent_id = response_data.get("id")
        if agent_id:
            print(f"\n✅ Agent created successfully!")
            print(f"🆔 Agent ID: {agent_id}")
        else:
            print("\n⚠️ Agent created but no ID found in response")
    except Exception as e:
        print(f"\n❌ Error parsing response: {e}")
else:
    print(f"\n❌ Failed to create agent. Status: {response.status_code}")

## Step 7: Invoke Your RAG Agent with Conversation

Now that your agent is created, you can test it using a conversational format with message history! This simulates a real conversation where the agent can reference previous exchanges.

**Conversation Format:**
- **`messages`**: Array of conversation turns with roles and content
- **`role`**: Either "user" (human) or "assistant" (AI agent)
- **`content`**: The actual message text
- **`filterValue`**: Additional filtering criteria (can be empty)

**How it works:**
1. Agent processes the entire conversation history
2. Uses context from previous messages to inform responses  
3. Searches relevant documents based on the latest user question
4. Provides contextually aware answers

**💡 Example Conversation:**
- User asks about sick leave policy
- Agent responds with policy details
- User follows up asking for more HR documents
- Agent understands the context and provides related information

In [ ]:
# Check if we have a valid agent ID from the previous step
print("🔍 Checking agent ID availability...")

if not agent_id:
    print("❌ No agent ID available. Please run Step 6 successfully first.")
    print("💡 You can also manually set agent_id = 'your-agent-id-here' if you know it.")
    print("🔧 To manually set agent ID, uncomment and modify the line below:")
    print("# agent_id = 'your-agent-id-here'")
else:
    print(f"✅ Found agent ID: {agent_id}")
    
    # Conversation payload with message history (from invoke_rag_agent notebook)
    conversation_payload = {
        "messages": [
            {
                "role": "user",
                "content": "What is the company's sick leave policy?"
            },
            {
                "role": "assistant", 
                "content": "The company's sick leave policy allows employees to take a certain number of sick days per year. Please refer to the employee handbook for specific details and eligibility criteria."
            },
            {
                "role": "user",
                "content": "Tell me more about HR docs?"
            }
        ],
        "filterValue": {}
    }
    
    # Make the invoke request using the latest version endpoint
    invoke_url = f"https://api.agents.ai.dev.experience.hyland.com/agent-platform/v1/agents/{agent_id}/versions/latest/invoke"
    
    print(f"🤖 Invoking agent with conversation: {agent_id}")
    print(f"🔗 Endpoint: {invoke_url}")
    print(f"💬 Conversation has {len(conversation_payload['messages'])} messages")
    print("📡 Making request...")
    
    try:
        invoke_response = requests.post(invoke_url, json=conversation_payload, headers=headers)
        print(f"📊 Response Status: {invoke_response.status_code}")
        
        if invoke_response.status_code != 200:
            print(f"⚠️ Response content: {invoke_response.text}")
            
    except Exception as e:
        print(f"❌ Error making request: {str(e)}")
        print("🔧 Check your network connection and try again")

## Step 8: Process Conversation Response

This cell processes the agent's response to the conversational query and displays it in a readable format.

**What to expect:**
- **Contextual Response**: The agent considers the full conversation history
- **Follow-up Awareness**: Understands "Tell me more about HR docs?" relates to the previous sick leave question
- **Document References**: Shows which documents were used to answer
- **Conversation Continuity**: Response builds on previous exchanges

**Response Elements:**
- **Agent's Answer**: The AI-generated response to the latest user message
- **Source Documents**: References to HR documents and policies used
- **Conversation Context**: How the agent interpreted the message history
- **Processing Metadata**: Information about retrieval and generation

In [ ]:
# Process the agent's conversation response
if 'invoke_response' in locals():
    if invoke_response.status_code == 200:
        try:
            response_data = invoke_response.json()
            
            print("🎉 SUCCESS! Here's your agent's conversational response:\n")
            print("=" * 70)
            
            # Show the conversation context
            print("💬 CONVERSATION CONTEXT:")
            for i, msg in enumerate(conversation_payload['messages'], 1):
                role_icon = "👤" if msg['role'] == 'user' else "🤖"
                print(f"{i}. {role_icon} {msg['role'].upper()}: {msg['content']}")
            
            print("\n" + "-" * 50)
            
            # Display the main response
            if 'response' in response_data:
                print("🤖 AGENT'S RESPONSE TO LATEST QUESTION:")
                print(f"'{conversation_payload['messages'][-1]['content']}'")
                print()
                print(response_data['response'])
                print("\n" + "-" * 50)
            
            # Display source information if available
            if 'sources' in response_data and response_data['sources']:
                print("\n📚 DOCUMENTS REFERENCED:")
                for i, source in enumerate(response_data['sources'], 1):
                    print(f"{i}. {source.get('title', 'Unknown Document')}")
                    if 'snippet' in source:
                        print(f"   📄 Excerpt: {source['snippet'][:120]}...")
                    if 'score' in source:
                        print(f"   🎯 Relevance: {source['score']:.2f}")
                print("\n" + "-" * 50)
            
            # Display processing metadata
            if 'metadata' in response_data:
                metadata = response_data['metadata']
                print(f"\n📊 PROCESSING DETAILS:")
                print(f"   💭 Conversation turns processed: {len(conversation_payload['messages'])}")
                if 'documents_retrieved' in metadata:
                    print(f"   📋 Documents retrieved: {metadata['documents_retrieved']}")
                if 'processing_time' in metadata:
                    print(f"   ⏱️ Processing time: {metadata['processing_time']}ms")
            
            print("=" * 70)
            
        except Exception as e:
            print(f"❌ Error processing response: {e}")
            print(f"Raw response: {invoke_response.text}")
    
    else:
        print(f"❌ Agent conversation failed!")
        print(f"Status Code: {invoke_response.status_code}")
        print(f"Error: {invoke_response.text}")
        
        # Provide helpful error messages
        if invoke_response.status_code == 404:
            print("\n💡 Troubleshooting:")
            print("   - Check if the agent ID is correct")
            print("   - Verify the agent was created successfully")
            print("   - Ensure the agent has been deployed")
        elif invoke_response.status_code == 401:
            print("\n💡 Troubleshooting:")
            print("   - Your access token may have expired")
            print("   - Re-run Step 3 to get a new token")

else:
    print("ℹ️ No conversation response to process. Run Step 7 first to invoke your agent.")

## Complete Workflow Summary

### 🎯 What You've Accomplished
You've successfully:
1. **Set up authentication** with the Hyland API
2. **Configured a RAG agent** with custom parameters
3. **Created the agent** on the platform
4. **Tested the agent** with real queries

### ✅ If Everything Worked
Your RAG agent is now ready for production use! You can:
- **Integrate it into applications** using the agent ID
- **Scale up document processing** by uploading more content
- **Monitor performance** through the Hyland dashboard
- **Modify configuration** by creating new agent versions

---

## Troubleshooting Guide

### 🔧 Agent Creation Issues (Step 6)

**Status 201: Success** ✅
- Agent created successfully
- Agent ID extracted and ready for use

**Status 400: Bad Request** ❌
- Check your payload configuration in Step 5
- Verify all required fields are present
- Ensure `llm_model_id` is valid

**Status 401: Unauthorized** ❌
- Verify your CLIENT_ID and CLIENT_SECRET in Step 2
- Check if credentials have expired
- Ensure you're using the correct environment

**Status 403: Forbidden** ❌
- Confirm your account has agent creation permissions
- Check if you're within usage limits

### 🤖 Agent Invocation Issues (Steps 7-8)

**Status 200: Success** ✅
- Query processed successfully
- Response contains agent's answer and sources

**Status 400: Bad Request** ❌
- Check your query format
- Ensure query is not empty
- Try a simpler question

**Status 404: Not Found** ❌
- Agent ID is incorrect or agent doesn't exist
- Verify Step 6 completed successfully
- Check if agent was deleted

**Status 401: Unauthorized** ❌
- Access token may have expired (tokens last ~1 hour)
- Re-run Step 3 to get a new token
- Check if headers are properly formatted

### 🔄 Testing Different Configurations

**To modify your agent:**
1. Go back to **Step 5** and change the `payload` dictionary
2. Re-run Step 6 to create a new agent
3. Test with Steps 7-8

**Common modifications:**
- Change `temperature` for more/less creative responses
- Adjust `limit` to retrieve more/fewer documents
- Modify `system_prompt` to change response style
- Try different `llm_model_id` options

### 🔄 Asking Multiple Questions

After completing all steps, you can:
1. **Rerun Steps 7-8** with different queries
2. **Change your question** in Step 7 and see different responses
3. **Compare responses** to understand your agent's capabilities

### 📚 Additional Resources
- [Hyland Agent Platform Documentation](https://docs.hyland.com)
- [RAG Agent Configuration Guide](https://docs.hyland.com/agents/rag)
- [API Reference](https://api.agents.ai.dev.experience.hyland.com/docs)
- [Best Practices for RAG Prompting](https://docs.hyland.com/agents/prompting)